С момента выпуска ChatGPT получил множество отзывов от пользователей, которые выражают свое мнение с помощью оценок и текстовых обзоров. Понимание настроений пользователей и факторов, определяющих удовлетворенность или неудовлетворенность, имеет решающее значение для улучшения продукта и улучшения пользовательского опыта.

# Import библиотек

In [1]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"
from textblob import TextBlob

# Набор данных

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Analysis-projects/chatgpt_reviews.csv")
df

,Review Id,Review,Ratings,Review Date
0,6fb93778-651a-4ad1-b5ed-67dd0bd35aac,good,5,2024-08-23 19:30:05
1,81caeefd-3a28-4601-a898-72897ac906f5,good,5,2024-08-23 19:28:18
2,452af49e-1d8b-4b68-b1ac-a94c64cb1dd5,nice app,5,2024-08-23 19:22:59
3,372a4096-ee6a-4b94-b046-cef0b646c965,"nice, ig",5,2024-08-23 19:20:50
4,b0d66a4b-9bde-4b7c-8b11-66ed6ccdd7da,"this is a great app, the bot is so accurate to...",5,2024-08-23 19:20:39
...,...,...,...,...
196722,462686ff-e500-413c-a6b4-2badc2e3b21d,Update 2023,5,2023-07-27 16:26:31
196723,f10e0d48-ecb6-42db-b103-46c0046f9be9,its grear,5,2023-09-23 16:25:18
196724,df909a49-90b5-4dac-9b89-c4bd5a7c2f75,Funtastic App,5,2023-11-08 13:57:14
196725,abe43878-973f-4e96-a765-c4af5c7f7b20,hi all,5,2023-07-25 15:32:57


## Анализ набора данных

**Названия столбцов**:


* Review Id - id отзыва
* Review - отзыв
* Ratings - оценка
* Review Date - дата отзыва



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196727 entries, 0 to 196726
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Review Id    196727 non-null  object
 1   Review       196721 non-null  object
 2   Ratings      196727 non-null  int64 
 3   Review Date  196727 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.0+ MB


In [4]:
df['Review Date'] = pd.to_datetime(df['Review Date'])

Все данные у нас нужного типа, кроме столбца с Датой. Поменяем тип у данного столбца

### Проверим df на дубли и пустые значения

#### Проверка на дубли

In [5]:
df.duplicated().sum()

2511

Посмотрим на наши дубли

In [6]:
df[df.duplicated()].sort_values(by='Review Id')

,Review Id,Review,Ratings,Review Date
79634,0009b10b-59f3-4c66-a68d-d330ad280a55,သဘောတွေ့ပါတယ်ခက်ဗျာ,5,2024-05-17 10:26:45
80290,0009b10b-59f3-4c66-a68d-d330ad280a55,သဘောတွေ့ပါတယ်ခက်ဗျာ,5,2024-05-17 10:26:45
80038,000ca1a5-ed93-4cf2-875e-6e8bb4941bf6,good,5,2024-05-17 03:54:56
80694,000ca1a5-ed93-4cf2-875e-6e8bb4941bf6,good,5,2024-05-17 03:54:56
79702,010b268a-fd6a-409c-9985-2c4bde998132,زبردست,5,2024-05-17 09:31:21
...,...,...,...,...
84213,ffc29c36-0cd3-4822-b364-9ca757bd6fd6,Shaktiman,5,2024-05-12 03:43:41
80649,ffc4c0c3-1cd9-4a2b-ab01-6fd55903bf38,fast,5,2024-05-17 04:42:26
79993,ffc4c0c3-1cd9-4a2b-ab01-6fd55903bf38,fast,5,2024-05-17 04:42:26
80512,ffd577a6-8fbd-4b57-8604-2615eb8253ec,I am using ChatGPT every day on my browsers on...,1,2024-05-17 07:11:34


In [7]:
df.drop_duplicates(inplace=True)

У нас в df есть 2511 полных дублей, как видно пользователь отправил отзыв несколько раз и система случайно задублила данные, поэтому бsло решено их удалить

#### Проверка на пустые значения

In [8]:
df.isnull().sum()

,0
Review Id,0
Review,6
Ratings,0
Review Date,0


In [9]:
df[df['Review'].isnull()].sort_values(by='Review Id')

,Review Id,Review,Ratings,Review Date
169192,4c07a17f-5c2a-489b-8136-49fae85a6cfa,NaN,5,2024-03-29 13:52:29
162561,527e0514-b444-4c59-b0a9-7f7774335ece,NaN,5,2024-04-04 18:56:52
35896,6d2f3aaa-babf-4105-9743-fa31462a526d,NaN,1,2024-07-11 18:15:05
169011,a16c4191-19e8-4cc2-b072-0118fae7b5f4,NaN,5,2024-03-23 23:03:54
49374,a6fc983e-8729-4d78-8cdf-25a47194932d,NaN,1,2024-06-24 06:20:22
155476,ffc32dd8-45b3-44a0-b8cf-8bd83ec5a621,NaN,1,2024-04-25 06:07:13


In [10]:
df['Review'] = df['Review'].astype(str).fillna('')

У нас есть 6 пустых отзывов, поэтому заполним их пустым текстом, т.к. нам важно будет их оценка

# Анализ тональности отзывов

## Распределение тональноти текста

In [11]:
#сделаем функцию, которая опознает тональность в тексте
def get_sentiment(review):
    sentiment = TextBlob(review).sentiment.polarity
    if sentiment > 0:
        return 'Positive'
    elif sentiment < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [12]:
df['Sentiment'] = df['Review'].apply(get_sentiment)

In [13]:
sentiment_distribution_1 = df['Sentiment'].value_counts()
sentiment_distribution_2 = df['Sentiment'].value_counts(normalize=True) * 100

In [14]:
sentiment_distribution = pd.concat([sentiment_distribution_1, sentiment_distribution_2], axis=1)
sentiment_distribution

,count,proportion
Sentiment,,
Positive,148219,76.316575
Neutral,37928,19.528772
Negative,8069,4.154653


Как можно увидеть большинство отзывов у нас положительные (76%). Посмотрим это отдельно на графике

In [15]:
fig = go.Figure(data=[go.Bar(
    x=sentiment_distribution_1.index,
    y=sentiment_distribution_1.values,
    marker_color=['green', 'gray', 'red'],
)])

fig.update_layout(
    title='Распределение тональности отзывов',
    xaxis_title='Тональность',
    yaxis_title='Количество отзывов',
    width=800,
    height=600
)

fig.show()

Посмотрим отдельно какие же отзывы популярны в каждой категории

## Популярные "положительные" отзывы

In [17]:
# возьмем отдельно столбец отзывы с положительной интонацией
positive_reviews = df[df['Sentiment'] == 'Positive']['Review']

# сделаем vectorizer, чтобы выделить популярные связки слов
vectorizer = CountVectorizer(ngram_range=(2, 3), stop_words='english', max_features=100)
X = vectorizer.fit_transform(positive_reviews)

phrase_counts = X.sum(axis=0)
phrases = vectorizer.get_feature_names_out()
phrase_freq = [(phrases[i], phrase_counts[0, i]) for i in range(len(phrases))]

phrase_freq = sorted(phrase_freq, key=lambda x: x[1], reverse=True)

phrase_df = pd.DataFrame(phrase_freq, columns=['Фразы', 'Частота'])

fig = px.bar(phrase_df,
             x='Частота',
             y='Фразы',
             orientation='h',
             title='Популярные "положительные" отзывы',
             labels={'Фразы': 'Фразы', 'Частота': 'Частота'},
             width=1000,
             height=600)

fig.update_layout(
    xaxis_title='Частота',
    yaxis_title='Фразы',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

Как можно видеть из популярных отзывов самый популярный отзыв это "great app", "amazing app", "ai app" и "excellent app". Соответственно можно сказать, что ChatGPT помог людям и они высказывают мнения об его полезности.



## Популярные "отрицательные" отзывы

In [19]:
# возьмем отдельно столбец отзывы с отрицательной интонацией
negative_reviews = df[df['Sentiment'] == 'Negative']['Review']

# выделим популярные связки слов
X_neg = vectorizer.fit_transform(negative_reviews)

phrase_counts_neg = X_neg.sum(axis=0)
phrases_neg = vectorizer.get_feature_names_out()
phrase_freq_neg = [(phrases_neg[i], phrase_counts_neg[0, i]) for i in range(len(phrases_neg))]

phrase_freq_neg = sorted(phrase_freq_neg, key=lambda x: x[1], reverse=True)

phrase_neg_df = pd.DataFrame(phrase_freq_neg, columns=['Фразы', 'Частота'])

fig = px.bar(phrase_neg_df,
             x='Частота',
             y='Фразы',
             orientation='h',
             title='Популярные "отрицательные" отзывы',
             labels={'Фразы': 'Фразы', 'Частота': 'Частота'},
             width=1000,
             height=600)

fig.update_layout(
    xaxis_title='Частота',
    yaxis_title='Фразы',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

Как можно видеть из популярных отзывов самый популярный отзыв это "bad app", "doesn work", "use app" и "web version". Соответственно можно сказать, что некоторым людям ChatGPT не очень помог, у кого-то он не работал, у кого-то произошла ошибка при использовании.

# Распространенные проблемы

Создадим словарь с определенными темами проблем и запишем популярные слова по ним

In [20]:
problem_keywords = {
    'Incorrect Answers': ['wrong answer', 'gives wrong', 'incorrect', 'inaccurate', 'wrong'],
    'App Performance': ['slow', 'lag', 'crash', 'bug', 'freeze', 'loading', 'glitch', 'worst app', 'bad app', 'horrible', 'terrible'],
    'User Interface': ['interface', 'UI', 'difficult to use', 'confusing', 'layout'],
    'Features Missing/Not Working': ['feature missing', 'not working', 'missing', 'broken', 'not available'],
    'Quality of Responses': ['bad response', 'useless', 'poor quality', 'irrelevant', 'nonsense']
}

problem_counts = {key: 0 for key in problem_keywords.keys()}

Посчитаем по каждой проблеме частоту отзывов и нарисуем график

In [25]:
for phrase, count in phrase_freq_neg:
    for problem, keywords in problem_keywords.items():
        if any(keyword in phrase for keyword in keywords):
            problem_counts[problem] += count
            break

problem_df = pd.DataFrame(list(problem_counts.items()), columns=['Проблема', 'Частота'])

In [26]:
fig = px.bar(problem_df,
             x='Частота',
             y='Проблема',
             orientation='h',
             title='Распространенные проблемы',
             labels={'Проблема': 'Проблема', 'Частота': 'Частота'},
             width=1000,
             height=600)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis_title='Частота',
    yaxis_title='Проблема',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

Основные проблемы по отзывам связаны у нас с Неправильными ответами и Работой платформы. В основном пользователи не смогли получить нужный ответ, к сожалению, ChatGPT не знает всего на свете.

# Динамика отзывов по времени

In [27]:
sentiment_over_time = df.groupby([df['Review Date'].dt.to_period('M'), 'Sentiment']).size().unstack(fill_value=0)

sentiment_over_time.index = sentiment_over_time.index.to_timestamp()

fig = go.Figure()

fig.add_trace(go.Scatter(x=sentiment_over_time.index, y=sentiment_over_time['Positive'],
                         mode='lines', name='Positive', line=dict(color='green')))
fig.add_trace(go.Scatter(x=sentiment_over_time.index, y=sentiment_over_time['Neutral'],
                         mode='lines', name='Neutral', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=sentiment_over_time.index, y=sentiment_over_time['Negative'],
                         mode='lines', name='Negative', line=dict(color='red')))

fig.update_layout(
    title='Sentiment Trends Over Time',
    xaxis_title='Date',
    yaxis_title='Number of Reviews',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title_text='Sentiment',
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)

fig.show()

По граыику можно сказать, что большинство пользователей начали массово оставлять отзывы с марта 2024, причем идет рост не только Положительных, но и Отрицательных отзывов

# NPS

Рассчитаем показатель NPS, чтобы посмотреть насколько сильно пользователи пользователи любят данное приложение

In [28]:
# для начала создадим категории, Промоутеры - это те, кто поставили 5 баллов, Нейтралы - 4 балла, остальные оценки Детракторы
df['NPS Category'] = df['Ratings'].apply(lambda x: 'Promoter' if x == 5 else ('Passive' if x == 4 else 'Detractor'))

# посчитаем доли для каждой категории
nps_counts = df['NPS Category'].value_counts(normalize=True) * 100

# посчитаем показатель с помощью формулы (NPS= %Promoters − %Detractors)
nps_score = nps_counts.get('Promoter', 0) - nps_counts.get('Detractor', 0)

nps_score

64.30829591794702

Показатель NPS для ChatGPT, основанный на оценках, представленных в наборе данных, составляет приблизительно 64,31. Это указывает на высокую вероятность того, что пользователи будут рекомендовать приложение другим, поскольку оценка выше 50, как правило, считается отличной.